In [7]:
import os
import rasterio
from rasterio.transform import from_origin
from rasterio.mask import mask
from shapely.geometry import box
from osgeo import gdal

# Clip the rasters
- Clip a raster with a reference raster one

In [12]:
def clip_raster(input_path, output_path, reference_raster):
    # Open the reference raster to get its bounds
    with rasterio.open(reference_raster) as ref_ds:
        reference_bounds = ref_ds.bounds

    # Create a geometry from the reference raster's bounds
    reference_geometry = box(reference_bounds.left, reference_bounds.bottom, reference_bounds.right, reference_bounds.top)

    """Prepare the name"""
    # Extract the filename without extension
    filename_no_ext = os.path.splitext(os.path.basename(input_path))[0]

    # Create the output filename with "_clipped" suffix
    output_filename = f"{filename_no_ext}_clipped.tif"
    output_path = os.path.join(output_folder, output_filename)

    # Open the input raster to be clipped
    with rasterio.open(input_path) as ds:
        # Clip the raster to the reference raster's geometry
        out_image, out_transform = mask(ds, shapes=[reference_geometry], crop=True)

        # Update the raster properties
        kwargs = ds.meta.copy()
        kwargs.update({
            'driver': 'GTiff',
            'transform': out_transform,
            'width': out_image.shape[2],
            'height': out_image.shape[1],
            'compress': "deflate",
            'tiled': True
        })

        # Create a new output raster with the clipped properties
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(out_image)

def process_rasters(input_folder, output_folder, reference_raster):
    # Iterate through raster files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            # Clip the raster and save a copy
            clip_raster(input_path, output_path, reference_raster)

In [13]:
# Set the paths and reference raster
input_folder = r"C:\Users\admin\Downloads\01_import\relative_magnitude_fragmentation"
output_folder = r"C:\Users\admin\Downloads\02_outputs"
# reference_raster = r"C:\Users\admin\Downloads\marcel\data_extraction\test_data\forest-condition_lower-bounds-reference-areas_year2000_EPSG3035.tif"
reference_raster = r"C:\Users\admin\Downloads\marcel\forest-condition_lower-bounds-reference-areas_year2000_EPSG3035.tif"

# Process rasters
process_rasters(input_folder, output_folder, reference_raster)

# assign the resolution

In [14]:
def reproject_raster(input_path, output_path):
    # Set the target projection (EPSG:3035)
    target_projection = 'EPSG:3035'

    # Set the desired width and height
    target_width = 66583
    target_height = 58832

    # Set the desired extent
    target_extent = [943600.0, 941700.0, 7601900.0, 6824900.0]
                     
    # Set the options for gdal.Warp
    options = gdal.WarpOptions(
        format="GTiff",
        width=target_width,
        height=target_height,
        dstSRS=target_projection,
        outputBounds=target_extent,
        # dstNodata=255,
        creationOptions=['COMPRESS=DEFLATE', 'TILED=YES']
        # resampleAlg=gdal.GRA_Bilinear  # You can choose a different resampling algorithm if needed
    )

    # Reproject the raster using gdal.Warp
    gdal.Warp(output_path, input_path, options=options)
    
    # gdal.Warp(output_path, input_path, creationOptions = ['COMPRESS=DEFLATE', 'TILED=YES'], 
    #           width=target_width, height=target_height, dstSRS=target_projection, outputBounds=target_extent)

def process_output_folder(input_folder, output_folder):
    # Iterate through raster files in the output folder
    for filename in os.listdir(input_folder):
        if filename.endswith("_clipped.tif"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename.replace("_clipped.tif","") + "_adapted.tif")
            print(output_path)
            # Reproject the raster and save a copy
            reproject_raster(input_path, output_path)

In [15]:
# Set the paths and target raster
input_folder = r"C:\Users\admin\Downloads\02_outputs"
output_folder = r"C:\Users\admin\Downloads\02_outputs\adapted"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process rasters in the output folder
process_output_folder(input_folder, output_folder)

C:\Users\admin\Downloads\02_outputs\adapted\Copernicus_combined_drought_indicator_avg_of_EU_v3_2012_adapted.tif
C:\Users\admin\Downloads\02_outputs\adapted\relative_magnitude_fragmentation_2000_merged_total_clip_3035_adapted.tif
